In [1]:
import Pkg; Pkg.add("VegaLite"); Pkg.add("PrettyTables")
using JuMP, HiGHS
using Plots;
using VegaLite  # to make some nice plots
using DataFrames, CSV, PrettyTables
ENV["COLUMNS"]=120; # Set so all columns of DataFrames and Matrices are displayed

   Resolving package versions...
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Manifest.toml`


In [4]:
### 3. Create solver function (dcopf)

In [14]:
#=
Function to solve DC OPF problem using IEEE test cases
Inputs:
    gen_info -- dataframe with generator info
    line_info -- dataframe with transmission lines info
    loads  -- dataframe with load info
=#
function dcopf_ieee(gens, lines, loads)
    DCOPF = Model(HiGHS.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets based on data
      # Set of generator buses
    G = gens.id #the way it defines gens are by looking at which bus it's connected to - list gens as unique elements then create another matrix showing which gens are associated with which node in the system
                #set to id instead of connnode
    
      # Set of all nodes
    N = sort(union(unique(lines.fromnode), 
            unique(lines.tonode)))  #gives 36 nodes 1...36
    
    #set of lines
    L = lines.id #each line has been assigned an id from 1...66
    
      # sets J_i and G_i will be described using dataframe indexing below

    # Define per unit base units for the system 
    # used to convert from per unit values to standard unit
    # values (e.g. p.u. power flows to MW/MVA)
    baseMVA = 100000000 # base MVA is 100 MVA for this system *LOOK INTO THIS FURTHER*
    
    # Decision variables   #ammended to include Ug
    @variables(DCOPF, begin
        GEN[G]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        THETA[N]         # voltage phase angle of bus
        FLOW[L]          # flows along each line
        U[g]             #Variable Ug
         
    end)
    
    set_binary(U[G])
    
    # Create slack bus with reference angle = 0; use bus 1 with generator
    fix(THETA[1],0)
                
    # Objective function
    @objective(DCOPF, Min, 
        sum(gens[g,:c1] * GEN[g] for g in G)
    )
    
    # old Supply demand balances
    #@constraint(DCOPF, cBalance[i in N], 
     #   sum(GEN[g] for g in gens[gens.connnode .== i,:connnode]) 
      #      + sum(load for load in loads[loads.connnode .== i,:demand]) 
       # == sum(FLOW[i,j] for j in lines[lines.fromnode .== i,:tonode])
    #)

    # demand balance
    @constraint(DCOPF, cSupBalance[i in N],
        sum((I[g,N] * GEN[g]) for g in gens[gens.connode .== i, :connnode]) 
        - sum((A[l, N] * FLOW[l]) for l in lines[lines.id .== i])
        == loads[loads.connnode .== i,:demand]
    )

    # Max generation constraint # AMMENDED 
    @constraint(DCOPF, cMaxGen[g in G],
              gens[g,:pgmin]*U[G] <= GEN[g] <= gens[g,:pgmax]*U[G])
    
    # Flow constraints on each branch #AMMENDED
    @constraint(DCOPF, cLineFlows[l in 1:nrow(lines)],
            FLOW[lines[l,:fromnode],lines[l,:tonode]] == 
            lines[l,:b] * sum(A[l,N] .* (THETA[lines[l,:fromnode]] - THETA[lines[l,:tonode]])) 
            
    )
    
    # line flow constraints AMMENDED
    @constraint(DCOPF, cLineLimits[l in 1:nrow(lines)], 
            -lines[l,:capacity] <= FLOW[lines[l,:fromnode],lines[l,:tonode]] <=
            lines[l,:capacity]
    ) 


    # Solve statement (! indicates runs in place)
    optimize!(DCOPF)

    # Output variables
    generation = DataFrame(
        node = gens.connnode,
        gen = value.(GEN).data[gens.connnode]
        )
    
    angles = value.(THETA).data
    
    #Ammmended to show flows on each line 
    flows = DataFrame(
        flow = lines.b .* sum((angles[lines.fromnode] .- 
                        angles[lines.tonode]) * A[L,N] ))
    
    # We output the marginal values of the demand constraints, 
    # which will in fact be the prices to deliver power at a given bus.
    prices = DataFrame(
        node = N,
        value = dual.(cBalance).data)
   
    # Return load payment
    loadpay = sum((dual.(cSupBalance)) * loads[loads.connnode .== i,:demand])
    
    #congestion cost
    congestc = -sum(FLOW[l] for l in load )

    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        angles,
        flows,
        prices,
        cost = objective_value(DCOPF),
        status = termination_status(DCOPF)
    )
end

dcopf_ieee (generic function with 1 method)

In [12]:
datadir = joinpath("testcase") 
gens = CSV.read(joinpath(datadir,"Gen36.csv"), DataFrame);
lines = CSV.read(joinpath(datadir,"Tran36_b_csv.csv"), DataFrame);
loads = CSV.read(joinpath(datadir,"Load36_csv.csv"), DataFrame);


# Rename all columns to lowercase (by convention)
for f in [gens, lines, loads]
    rename!(f,lowercase.(names(f)))
end

# create generator ids 
gens.id = 1:nrow(gens);

# create line ids 
lines.id = 1:nrow(lines);
# add set of rows for reverse direction with same parameters

#lines2 = copy(lines)
#lines2.f = lines2.fromnode
#lines2.fromnode = lines.tonode
#lines2.tonode = lines.fromnode
#lines2 = lines2[:,names(lines)]
#append!(lines,lines2)

# calculate simple susceptance, ignoring resistance as earlier 
lines.b = 1 ./ lines.reactance

# keep only a single time period
loads = loads[:,["connnode","interval-1_load"]]
rename!(loads,"interval-1_load" => "demand");

lines

Row,fromnode,tonode,resistance,reactance,contingencymarked,capacity,id,b
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64
1,1,2,0.00266925,0.0323779,1,400000,1,30.8853
2,1,5,0.00287887,0.0286089,1,400000,2,34.9541
3,2,3,0.00131306,0.0190229,1,400000,3,52.5681
4,2,6,0.000803188,0.0149712,1,400000,4,66.795
5,3,10,0.000399063,0.00548875,1,400000,5,182.191
6,4,5,0.00114981,0.0183772,1,400000,6,54.4153
7,4,6,0.00209787,0.0318498,1,400000,7,31.3974
8,4,14,0.00220469,0.0290109,1,400000,8,34.4698
9,5,6,0.000706313,0.00961025,1,400000,9,104.056


In [15]:
solution = dcopf_ieee(gens, lines, loads);

LoadError: UndefVarError: `g` not defined